In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-08-31T08:14:53.001677-04:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.28.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-37-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os

In [5]:
%watermark --iversions

pandas : 1.4.3
sklearn: 1.1.2
numpy  : 1.22.4



In [6]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [7]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

In [10]:
pca = PCA(n_components=0.9, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

In [11]:
X.shape

(60000, 87)

In [12]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [13]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.9839166666666667
Fitting fold 2
Fold accuracy: 0.9815833333333334
Fitting fold 3
Fold accuracy: 0.9815833333333334
Fitting fold 4
Fold accuracy: 0.98075
Fitting fold 5
Fold accuracy: 0.9814166666666667
CPU times: user 10min 48s, sys: 2min 47s, total: 13min 36s
Wall time: 2min 34s


In [14]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.98185


In [14]:
0.98415

0.98415

In [15]:
preds = np.argmax(test_preds, axis=1)
preds

array([7, 2, 1, ..., 4, 5, 6])

In [16]:
Y_test

array([7, 2, 1, ..., 4, 5, 6])

In [17]:
accuracy_score(Y_test, preds)

0.9828

In [19]:
0.9828

0.9828

In [18]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.9845
Fitting fold 2
Fold accuracy: 0.9836666666666667
Fitting fold 3
Fold accuracy: 0.9833333333333333
Fitting fold 4
Fold accuracy: 0.9835
Fitting fold 5
Fold accuracy: 0.9820833333333333
0.9834166666666667
0.9859
CPU times: user 8min 10s, sys: 2min 55s, total: 11min 6s
Wall time: 2min


In [20]:
0.9859

0.9859

In [21]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.81, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 46)
Fitting fold 1
Fold accuracy: 0.9849166666666667
Fitting fold 2
Fold accuracy: 0.9838333333333333
Fitting fold 3
Fold accuracy: 0.9836666666666667
Fitting fold 4
Fold accuracy: 0.98425
Fitting fold 5
Fold accuracy: 0.9819166666666667
0.9837166666666667
0.9851
CPU times: user 8min 10s, sys: 2min 59s, total: 11min 9s
Wall time: 2min 1s


In [22]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=48, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 48)
Fitting fold 1
Fold accuracy: 0.9846666666666667
Fitting fold 2
Fold accuracy: 0.9836666666666667
Fitting fold 3
Fold accuracy: 0.9833333333333333
Fitting fold 4
Fold accuracy: 0.9840833333333333
Fitting fold 5
Fold accuracy: 0.9825
0.98365
0.9855
CPU times: user 8min 40s, sys: 2min 50s, total: 11min 31s
Wall time: 2min 7s


In [23]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=50, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 50)
Fitting fold 1
Fold accuracy: 0.9850833333333333
Fitting fold 2
Fold accuracy: 0.9833333333333333
Fitting fold 3
Fold accuracy: 0.9834166666666667
Fitting fold 4
Fold accuracy: 0.9849166666666667
Fitting fold 5
Fold accuracy: 0.9825
0.98385
0.985
CPU times: user 8min 27s, sys: 2min 49s, total: 11min 17s
Wall time: 2min 6s


In [24]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=42, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 42)
Fitting fold 1
Fold accuracy: 0.9853333333333333
Fitting fold 2
Fold accuracy: 0.9835833333333334
Fitting fold 3
Fold accuracy: 0.9836666666666667
Fitting fold 4
Fold accuracy: 0.9834166666666667
Fitting fold 5
Fold accuracy: 0.9825833333333334
0.9837166666666667
0.9856
CPU times: user 7min 58s, sys: 2min 54s, total: 10min 53s
Wall time: 1min 59s


In [25]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=40, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 40)
Fitting fold 1
Fold accuracy: 0.9858333333333333
Fitting fold 2
Fold accuracy: 0.9836666666666667
Fitting fold 3
Fold accuracy: 0.9839166666666667
Fitting fold 4
Fold accuracy: 0.98275
Fitting fold 5
Fold accuracy: 0.9820833333333333
0.98365
0.985
CPU times: user 7min 50s, sys: 2min 50s, total: 10min 40s
Wall time: 1min 56s


In [26]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=12, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.9844166666666667
Fitting fold 2
Fold accuracy: 0.9835833333333334
Fitting fold 3
Fold accuracy: 0.9838333333333333
Fitting fold 4
Fold accuracy: 0.9838333333333333
Fitting fold 5
Fold accuracy: 0.9818333333333333
0.9835
0.9858
CPU times: user 8min 13s, sys: 2min 51s, total: 11min 4s
Wall time: 2min 1s


In [27]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='rbf',C=8, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.9848333333333333
Fitting fold 2
Fold accuracy: 0.9835833333333334
Fitting fold 3
Fold accuracy: 0.9833333333333333
Fitting fold 4
Fold accuracy: 0.9838333333333333
Fitting fold 5
Fold accuracy: 0.9820833333333333
0.9835333333333334
0.9858
CPU times: user 8min 14s, sys: 2min 52s, total: 11min 7s
Wall time: 2min 1s


In [28]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='poly', degree=3, C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.98275
Fitting fold 2
Fold accuracy: 0.9784166666666667
Fitting fold 3
Fold accuracy: 0.97925
Fitting fold 4
Fold accuracy: 0.9788333333333333
Fitting fold 5
Fold accuracy: 0.9776666666666667
0.9793833333333334
0.9804
CPU times: user 11min 28s, sys: 2min 42s, total: 14min 10s
Wall time: 2min 27s


In [31]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='poly', degree=6, C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.96225
Fitting fold 2
Fold accuracy: 0.9611666666666666
Fitting fold 3
Fold accuracy: 0.95725
Fitting fold 4
Fold accuracy: 0.9586666666666667
Fitting fold 5
Fold accuracy: 0.9610833333333333
0.9600833333333333
0.9578
CPU times: user 21min 9s, sys: 3min 37s, total: 24min 47s
Wall time: 4min


In [32]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='poly', degree=9, C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.8923333333333333
Fitting fold 2
Fold accuracy: 0.89025
Fitting fold 3
Fold accuracy: 0.8868333333333334
Fitting fold 4
Fold accuracy: 0.8859166666666667
Fitting fold 5
Fold accuracy: 0.8880833333333333
0.8886833333333334
0.8921
CPU times: user 36min 54s, sys: 3min 49s, total: 40min 43s
Wall time: 6min 38s


In [33]:
%%time
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

X = train[train.columns[1:]].values/255
X_test = test[test.columns[1:]].values/255
Y = train.label.values
Y_test = test.label.values

pca = PCA(n_components=0.8, whiten=True)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

print(X.shape)

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(kernel='poly', degree=2, C=10, probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()
    
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))


preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

(60000, 44)
Fitting fold 1
Fold accuracy: 0.9761666666666666
Fitting fold 2
Fold accuracy: 0.9754166666666667
Fitting fold 3
Fold accuracy: 0.9748333333333333
Fitting fold 4
Fold accuracy: 0.9751666666666666
Fitting fold 5
Fold accuracy: 0.9729166666666667
0.9749
0.9809
CPU times: user 10min 12s, sys: 2min 38s, total: 12min 50s
Wall time: 2min 12s
